In [9]:
import pandas as pd

# just train.csv not thinking about others. 
train_df = pd.read_csv("./train.csv")

y_actual = train_df['label'].to_numpy()
train_df.pop('label')
x_actual = train_df.to_numpy()

In [14]:
import torch

x_actual = torch.tensor(x_actual, dtype=torch.float32, requires_grad= False)
y_actual = torch.tensor(y_actual, dtype=torch.int64  , requires_grad= False)

# mapping to x_actual to y_actual. combining them together. 
for example in zip(x_actual, y_actual):
    print(example[0].shape, example[1].shape)
    break

dataset_dict = {'x_actual': x_actual, 'y_actual': y_actual}

try:
    dataset_dict[5]['x_actual']
    dataset_dict[5]
except RunTimeError:
    pass 


/var/folders/yc/sdgc0ftj32z_ctpgsn9ynz440000gn/T/ipykernel_9332/2561250100.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_actual = torch.tensor(x_actual, dtype=torch.float32, requires_grad= False)
/var/folders/yc/sdgc0ftj32z_ctpgsn9ynz440000gn/T/ipykernel_9332/2561250100.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_actual = torch.tensor(y_actual, dtype=torch.int64  , requires_grad= False)


torch.Size([784]) torch.Size([])


In [19]:
dir(torch.utils.data)

['BatchSampler',
 'ChainDataset',
 'ConcatDataset',
 'DFIterDataPipe',
 'DataChunk',
 'DataLoader',
 'Dataset',
 'DistributedSampler',
 'IterDataPipe',
 'IterableDataset',
 'MapDataPipe',
 'RandomSampler',
 'Sampler',
 'SequentialSampler',
 'Subset',
 'SubsetRandomSampler',
 'TensorDataset',
 'WeightedRandomSampler',
 '_DatasetKind',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_utils',
 'argument_validation',
 'dataloader',
 'datapipes',
 'dataset',
 'default_collate',
 'default_convert',
 'distributed',
 'functional_datapipe',
 'get_worker_info',
 'graph',
 'graph_settings',
 'guaranteed_datapipes_determinism',
 'non_deterministic',
 'random_split',
 'runtime_validation',
 'runtime_validation_disabled',
 'sampler']

In [21]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader

class MNISTModel(torch.nn.Module):
    def __init__(self):
        super(MNISTModel, self).__init__()
        # Define the model architecture
        self.conv1 = torch.nn.Conv2d(1, 32, 3, 1, padding=1)
        self.conv2 = torch.nn.Conv2d(32, 64, 3, 1, padding=1)
        self.fc1 = torch.nn.Linear(64 * 7 * 7, 128)
        self.fc2 = torch.nn.Linear(128, 10)

    def forward(self, x):
        # Forward pass through the model
        x = self.conv1(x)
        x = self.conv2(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

def train(model, train_loader, optimizer, loss_fn, epoch):
    # Train the model for one epoch
    model.train()
    for i, data in enumerate(train_loader):
        images, labels = data['image'], data['label']

        # Forward pass
        outputs = model(images)

        # Calculate the loss
        loss = loss_fn(outputs, labels)

        # Backward pass and update the parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print the loss every 1000 iterations
        if i % 1000 == 0:
            print(f'Epoch: {epoch + 1}, Iteration: {i}, Loss: {loss.item()}')

def evaluate(model, test_loader, loss_fn):
    # Evaluate the model on the test set
    model.eval()
    with torch.no_grad():
        total_loss = 0
        correct = 0
        total = 0
        for i, data in enumerate(test_loader):
            images, labels = data['image'], data['label']
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)

            # Calculate the loss
            loss = loss_fn(outputs, labels)

            # Update the total loss and accuracy
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    # Calculate the average loss and accuracy
    avg_loss = total_loss / total
    accuracy = correct / total

    return avg_loss, accuracy

if __name__ == '__main__':
    # Load the MNIST dataset from Hugging Face Datasets
    dataset = load_dataset('mnist')

    # Split the dataset into train and test sets
    train_dataset, test_dataset = dataset['train'], dataset['test']

    # Create PyTorch Dataloader objects for the train and test sets
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Define the model and optimizer
    model = MNISTModel()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Define the loss function
    loss_fn = torch.nn.CrossEntropyLoss()

    # Train the model for 10 epochs
    for epoch in range(10):
        train(model, train_loader, optimizer, loss_fn, epoch)

    # Evaluate the model on the test set
    avg_loss, accuracy = evaluate(model, test_loader, loss_fn)
    print(f'Test loss: {avg_loss:.4f}, Test accuracy: {accuracy:.4f}')


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.PngImagePlugin.PngImageFile'>

In [22]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load the MNIST dataset
dataset = load_dataset('mnist')

# Define the model architecture
class MNISTModel(torch.nn.Module):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 32, 3, 1, padding=1)
        self.conv2 = torch.nn.Conv2d(32, 64, 3, 1, padding=1)
        self.fc1 = torch.nn.Linear(64 * 7 * 7, 128)
        self.fc2 = torch.nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = torch.nn.functional.relu(x)
        x = torch.nn.functional.max_pool2d(x, 2)
        x = self.conv2(x)
        x = torch.nn.functional.relu(x)
        x = torch.nn.functional.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = torch.nn.functional.relu(x)
        x = self.fc2(x)
        return x

# Define the loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Create data loaders for the training and test sets
train_loader = DataLoader(dataset['train'], batch_size=32, shuffle=True)
test_loader = DataLoader(dataset['test'], batch_size=32, shuffle=False)

# Train the model for 10 epochs
model = MNISTModel()
for epoch in range(10):
    train(model, train_loader, optimizer, loss_fn, epoch)
    evaluate(model, test_loader, loss_fn)


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.PngImagePlugin.PngImageFile'>